#### Parallelization - Agentic Design Pattern

In [ ]:
import os

from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import colored
from dotenv import load_dotenv
from pydantic import BaseModel

import json
from rich.pretty import pprint
from uuid import uuid4

In [ ]:
load_dotenv()

In [ ]:
host = os.environ["TOGETHER_URL"]
together_api_key = os.environ["TOGETHER_API_KEY"]

client = LlamaStackClient(
    base_url=host,
    provider_data={
        "tavily_search_api_key": os.getenv("TAVILY_SEARCH_API_KEY"),
        "together_api_key": together_api_key,
    },
)

In [ ]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are a helpful assistant.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from typing import List


In [ ]:

worker_agent_config = {
    **base_agent_config,
    "instructions": """You are a helpful assistant that can analyze the impact of market changes on stakeholders.
    Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.
    """,
}

In [ ]:
def create_worker_task(task: str):
    worker_agent = Agent(client, **worker_agent_config)
    worker_session_id = worker_agent.create_session(session_name=f"worker_agent_{uuid4()}")
    task_response = worker_agent.create_turn(
        messages=[{"role": "user", "content": task}],
        stream=False,
        session_id=worker_session_id,
    )
    return {
        "worker_agent": worker_agent,
        "task_response": task_response.output_message.content,
    }

In [ ]:
def parallelization_workflow(tasks: List[str]):
    if isinstance(client, LlamaStackClient):
        with ThreadPoolExecutor(max_workers=len(tasks)) as executor:
            futures = [executor.submit(create_worker_task, task) for task in tasks]
            results = [future.result() for future in futures]
            
            return results
    else:
        results = []
        
        for task in tasks:
            result = create_worker_task(task)
            results.append(result)
            
        return results


In [ ]:
stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

In [ ]:
results = parallelization_workflow(stakeholders)

for i, result in enumerate(results):
    print(f"========= Stakeholder {i+1}: =========")
    print(result["task_response"])
    print("\n")

In [ ]:
for i, result in enumerate(results):
    print(f"========= Worker Agent {i+1}: =========")
    
    session_response = client.agents.session.retrieve(
        session_id=result["worker_agent"].session_id, agent_id=result["worker_agent"].agent_id)
    
    pprint(session_response.to_dict())
